In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U albumentations --user

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/SPACENET7/Multi-temporal.building.tracker/codes

In [ ]:
!pip install rasterio
!pip install patchify
!pip install geopandas

In [6]:
# Imports

import sys
import subprocess
import pkg_resources

required = {'opencv-python'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing], stdout=None)
    
import matplotlib.pyplot as plt
    
from patchify import patchify, unpatchify
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from torchvision import datasets, transforms, models

import torchvision.transforms as T
from pathlib import Path
import pathlib
from PIL import Image
import pandas as pd
import numpy as np
import os

import matplotlib

# Albumentations

import albumentations as A
from albumentations.pytorch import ToTensorV2

from tqdm import tqdm
from model import UNET

from data_loader import Dataloader_trdp
from utils import (
    load_checkpoint,
    load_checkpoint_pspnetlite,
    save_checkpoint,
    #get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
    predict,
    store_predictions,
    store_predictions_with_patching,
    store_predictions_unet_improved
)

# Clean graphics memory
import gc
gc.collect()
torch.cuda.empty_cache()

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [7]:
# Hyperparameters
in_channel = 3
num_classes = 1
learning_rate = 1e-4
batch_size = 1
num_epochs = 1
chip_dimension = 256
LOAD_MODEL = False
TRAIN = True
TEST = False
SAVE = False
# define threshold to filter weak predictions
THRESHOLD = 0.5
PREDICT = True

filename = "unet_23_dic_draft_dataset_customized_unet.pth" # my_checkpoint.pth.tar

In [8]:
% cd /content/drive/MyDrive/Colab Notebooks/SPACENET7/Multi-temporal.building.tracker/codes

UsageError: Line magic function `%` not found.


In [9]:
'''

train_dir = Path('/content/drive/MyDrive/Colab Notebooks/SPACENET7/DATASET/dataset_pruebas/train')
test_dir = Path('/content/drive/MyDrive/Colab Notebooks/SPACENET7/DATASET/dataset_pruebas/validation')


root_dir  = train_dir
csv_file = Path('/content/drive/MyDrive/Colab Notebooks/SPACENET7/DATASET/output_csvs_dataset_prueba/df_train_untidy.csv')
csv_file_test = Path('/content/drive/MyDrive/Colab Notebooks/SPACENET7/DATASET/output_csvs_dataset_prueba/df_test_untidy.csv')



'''

#TRAINING
#train_dir = Path('../../DATASET/archive/train_final')
#csv_file = Path('./output_csvs_all/df_train_untidy.csv')
train_dir  = Path('../../DATASET/dataset_pruebas/train')
csv_file = Path('./registros/output_csvs_dataset_prueba/df_train_untidy.csv')

#TESTING
test_dir  = Path('../../DATASET/dataset_pruebas/validation')
csv_file_test = Path('./registros/output_csvs_dataset_prueba/df_test_untidy.csv')
#test_dir  = Path('../../DATASET/archive/val_final')
#csv_file = Path('./registros/output_csvs_dataset_prueba/df_train_untidy.csv')

df = pd.read_csv(csv_file)
df_test = pd.read_csv(csv_file_test)
# Reconstructiontest_loader
img_size = 1024
# Chip size given batch_size
chip_dim = ((img_size -1)//batch_size + 1)*2
# number of Columns per chip
columns = img_size / chip_dim
# Needed patches to reconstruct original image
patches_total = int(columns**2)

###############################################################################

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
A.Normalize(mean=mean,std=std)
#A.Rotate(limit=(-360, 360), interpolation=4, border_mode=4,p=1),


transform = A.Compose(
    [
        #A.Resize(height=256, width=256),
        
        A.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        #A.RandomRotate90(p=1.0),
        ToTensorV2()
    ]
)

train_set   = Dataloader_trdp(root_dir=train_dir,csv_file=csv_file,chip_dimension=chip_dimension,transform=transform)

testing_set = Dataloader_trdp(root_dir=test_dir,csv_file=csv_file_test,chip_dimension=chip_dimension,transform=transform)

#train_set, val_set = torch.utils.data.random_split(train_set, [round(0.7*len(train_set)),round(0.3*len(train_set))])

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle = False)
#val_loader   = DataLoader(dataset = val_set, batch_size=batch_size, shuffle = False)
test_loader  = DataLoader(dataset = testing_set, batch_size=batch_size, shuffle = False)

print(f"Train : {len(train_loader)} - Test: {len(test_loader)}")



Train : 1504 - Test: 624


In [10]:
train_transform_train = A.Compose(
        [
            #A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

transform_test = A.Compose(
    [
        A.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        ToTensorV2()
    ]


SyntaxError: unexpected EOF while parsing (1025418016.py, line 20)

In [11]:
import torch as F
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

from torch.nn.modules.padding import ReplicationPad2d

def double_block(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=3),
        nn.ReLU(inplace = True),
        nn.Conv2d(out_c, out_c, kernel_size=3),
        nn.ReLU(inplace = True),
        
        )
    return conv
    

class unet_bas(nn.Module):
    def __init__(self):
        super(unet_bas, self).__init__()
        
        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride = 2)
        self.contractor_1 = double_block(3, 64)
        self.contractor_2 = double_block(64, 128)
        self.contractor_3 = double_block(128, 256)
        self.contractor_4 = double_block(256, 512)
        self.contractor_5 = double_block(512, 1024)
        
        # Decoder
        self.up_trans_1 = nn.ConvTranspose2d(in_channels = 1024, out_channels=512, kernel_size = 2, stride=2)
        self.up_conv_1 = double_block(1024, 512)
        
                
        self.up_trans_2 = nn.ConvTranspose2d(in_channels = 512, out_channels=256, kernel_size = 2, stride=2)
        self.up_conv_2 = double_block(512, 256)
        
                
        self.up_trans_3 = nn.ConvTranspose2d(in_channels = 256, out_channels=128, kernel_size = 2, stride=2)
        self.up_conv_3 = double_block(256, 128)
        
                
        self.up_trans_4 = nn.ConvTranspose2d(in_channels = 128, out_channels=64, kernel_size = 2, stride=2)
        self.up_conv_4 = double_block(128, 64)
        
        self.out = nn.Conv2d(in_channels = 64, out_channels = 1, kernel_size = 1)
        
    def forward(self, image):
        # Encoder
        # image (1, 3, 256, 256)
        x1 = self.contractor_1(image) # (1, 64, 252, 252)
        x2 = self.max_pool_2x2(x1) #(1, 128, 126, 126)
        x3 = self.contractor_2(x2)# (1, 128, 122, 122)
        x4 = self.max_pool_2x2(x3)# (1, 128, 61, 61)
        x5 = self.contractor_3(x4)# (1, 256, 57, 57)
        x6 = self.max_pool_2x2(x5)# (1, 256, 28, 28) -> 28.5
        x7 = self.contractor_4(x6)# 
        x8 = self.max_pool_2x2(x7)
        x9 = self.contractor_5(x8)
        
        #Decoder

        x = self.up_trans_1(x9)
        y = crop_img(x7, x)
        if y.shape != x.shape:
        	y = TF.resize(y, x.shape[2:])
        x = self.up_conv_1(torch.cat([x,y],1))
        
        x = self.up_trans_2(x)
        y = crop_img(x5, x)#torch.Size([1, 256, 25, 25])
        if y.shape != x.shape:
        	y = TF.resize(y, x.shape[2:])
        x = self.up_conv_2(torch.cat([x,y],1))
        
        x = self.up_trans_3(x)
        y = crop_img(x3, x)
        if y.shape != x.shape:
        	y = TF.resize(y, x.shape[2:])
        x = self.up_conv_3(torch.cat([x, y],1))
        
        
        x = self.up_trans_4(x)
        y = crop_img(x1, x)
        if y.shape != x.shape:
        	y = TF.resize(y, x.shape[2:])
        x = self.up_conv_4(torch.cat([x,y],1))
        
        # machetazo
        
        x = TF.resize(x, (256,256))
        x = self.out(x)
        
        return x
def crop_img(tensor, target_tensor):
	'''
	From original paper: take image of size 64 from 4th stage and crop it to fit with
	upconvoluted image of size 56. 
	e.g.
	target_size = 56
	tensor_size = 64
	delta = 8 // 2 # 4 
	
	#output
	tensor[:,:,delta:tensor_size-delta, delta:tensor_size-delta]
	tensor[:,:,4:64-4, 4:64-4]
	tensor[:,:,4:60, 4:60]
	
	# as a result, cropping is centered
	
	'''
	target_size = target_tensor.size()[2]
	
	tensor_size = tensor.size()[2]
	
	delta = tensor_size - target_size

	delta = delta//2
	
	x = tensor[:,:,delta:tensor_size-delta, delta:tensor_size-delta]
	return  x
        

        

test_mask = torch.randn((1, 3, 256, 256)) # RGB image
    ## MODEL 1
    #model = UNET(in_channels=1, out_channels=1)v
    #model = UNET_manual(3, 1)
model = unet_bas()
preds = model(test_mask)
print(preds.size())

torch.Size([1, 1, 256, 256])


In [ ]:

import torchvision.transforms.functional as TF
import random
import numpy as np
    
from torch.nn.modules.loss import _Loss


print('PyTorch version:', torch.__version__)


# https://github.com/SpaceNetChallenge/SpaceNet_SAR_Buildings_Solutions/blob/6a9c3962d987d985384d0d41a187f5fbfadac82c/2-MaksimovKA/train/losses.py#L14 
class FocalDiceLoss(torch.nn.Module):
    def __init__(self, coef_focal=1.0, coef_dice=1.0, weights=(1.0, 0.1, 0.5)):
        super().__init__()
       
        self.dice_loss = DiceLoss()
        self.focal_loss = FocalLoss()
        self.weights = weights

        self.coef_focal = coef_focal
        self.coef_dice = coef_dice

    def forward(self, outputs, targets):
        loss = 0.0

        for i in range(outputs.shape[1]):
            dice = self.weights[i]*self.dice_loss(outputs[:, i, ...], targets[:, i, ...])
            focal = self.weights[i]*self.focal_loss(outputs[:, i, ...], targets[:, i, ...])
            loss += self.coef_dice * dice + self.coef_focal * focal

        return loss

    
    
class FocalLoss(_Loss):

    def __init__(self, ignore_index=255, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ignore_index = ignore_index


    def forward(self, y_pred, y_true):
        """
        :param y_pred: NxCxHxW
        :param y_true: NxCxHxW
        :return: scalar
        """
        y_pred = y_pred.sigmoid()
        gamma = self.gamma
        ignore_index = self.ignore_index

        outputs = y_pred.contiguous()
        targets = y_true.contiguous()
        eps = 1e-8
        non_ignored = targets.view(-1) != ignore_index
        targets = targets.view(-1)[non_ignored].float()
        outputs = outputs.contiguous().view(-1)[non_ignored]
        outputs = torch.clamp(outputs, eps, 1. - eps)
        targets = torch.clamp(targets, eps, 1. - eps)
        pt = (1 - targets) * (1 - outputs) + targets * outputs
        return (-(1. - pt) ** gamma * torch.log(pt)).mean()


class DiceLoss(_Loss):

    def __init__(self, per_image=False):
        super(DiceLoss, self).__init__()
        self.per_image = per_image

    def forward(self, y_pred, y_true):
        """
        :param y_pred: NxCxHxW
        :param y_true: NxCxHxW
        :return: scalar
        """
        per_image = self.per_image
        y_pred = y_pred.sigmoid()
        
        batch_size = y_pred.size()[0]
        eps = 1e-5
        if not per_image:
            batch_size = 1
        
        dice_target = y_true.contiguous().view(batch_size, -1).float()
        dice_output = y_pred.contiguous().view(batch_size, -1)
        intersection = torch.sum(dice_output * dice_target, dim=1)
        union = torch.sum(dice_output, dim=1) + torch.sum(dice_target, dim=1) + eps
        loss = (1 - (2 * intersection + eps) / union).mean()

        return loss

class configuration:
    def __init__(self):
        self.experiment_name = "trdp1.001"
        self.pre_load = "True" ## Load dataset in memory
        self.pre_trained = "True"
        self.num_classes = num_classes
        self.ignore_label = 255
        self.lr = learning_rate  # 0.001 if pretrained. 0.1 if scratch
        self.M = [] ##If training from scratch, reduce learning rate at some point
        self.batch_size = batch_size  # Training batch size
        self.test_batch_size = 4  # Test batch size
        self.epoch = num_epochs ## Number of epochs
        self.train_root = "./VOC"
        self.download = False
        self.seed = 271828


## Create arguments object
args = configuration()

# Make sure to enable GPU acceleration!
device = "cuda" if torch.cuda.is_available() else "cpu"


# Set random seed for reproducability
torch.backends.cudnn.deterministic = True  # fix the GPU to deterministic mode
torch.manual_seed(args.seed)  # CPU seed
torch.cuda.manual_seed_all(args.seed)  # GPU seed
random.seed(args.seed)  # python seed for image transformation
np.random.seed(args.seed)


def _fast_hist(label_pred, label_true, num_classes):
    mask = (label_true >= 0) & (label_true < num_classes)
    hist = np.bincount(
        num_classes * label_true[mask].astype(int) +
        label_pred[mask], minlength=num_classes ** 2).reshape(num_classes, num_classes)
    return hist

def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    """Fast enough iou calculation function"""
    SMOOTH = 1e-6
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    #outputs = outputs.detach()
    #labels = labels.detach()
    
    intersection = (outputs & labels).float().sum((1, 2))
    union = (outputs | labels).float().sum((1, 2))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded.mean() # to get a batch averageimport pdb

def iou_fun(im1, im2, empty_score=1.0):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    union = np.logical_or(im1, im2)
    im_sum = union.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return intersection.sum() / im_sum
def train_SemanticSeg(args, model, device, train_loader, optimizer, epoch, criterion):
    # switch to train mode
    model.train()

    train_loss = []
    counter = 1

    gts_all, predictions_all = [], []

    for batch_idx, (data) in enumerate(train_loader):
        
        images = data["raster_diff"].float()
        
        mask = data["mask_diff"].float().squeeze()
        
        images, mask = images.to(device), mask.to(device)

        
        if torch.cuda.is_available():
            model.cuda()
            
        
        #Forward pass
        #print(images.size())
        outputs = model(images).squeeze()
        #print(outputs.size())
                
        #Aggregated per-pixel loss
        #loss = criterion(outputs, mask)
        
        loss = criterion(outputs.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0).unsqueeze(0))
        train_loss.append(loss.item())

        iou = iou_fun(mask.data.cpu().numpy(), outputs.data.cpu().numpy())#iou_pytorch(mask.unsqueeze(0).int(), outputs.int())
        
        # compute gradient and do SGD step
        optimizer.zero_grad()

        loss.backward()
        
        optimizer.step()
        
        if counter % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Learning rate: {:.6f}'.format(
                epoch, int(counter * len(images)), len(train_loader.dataset),
                100. * counter / len(train_loader), loss.item(),
                optimizer.param_groups[0]['lr']))
            print(f"Iou: {iou}")
        counter = counter + 1
        
    return sum(train_loss) / len(train_loss)#, mean_iu
        
def testing(args, model, device, test_loader, criterion):

    # switch to train mode
    model.eval()
    loss_per_batch = []
    test_loss = 0
    
    gts_all, predictions_all = [], []
    with torch.no_grad():
        for batch_idx, (data) in enumerate(test_loader):
            
            print(f" {batch_idx}/{len(test_loader)} ")
            if batch_idx <= 992:
            
                images = data["raster_diff"].float()
                mask = data["mask_diff"].float()#type(torch.LongTensor)
                images, mask = images.to(device), mask.to(device).squeeze()
                #Forward pass
                    
                if torch.cuda.is_available():
                    model.cuda()
                outputs = model(images).squeeze()
                #outputs = outputs.clone().detach().cpu().numpy()
        #        outputs = outputs.cpu().numpy()
                outputs = ((outputs - outputs.min())/(outputs.max()-outputs.min()))
                #outputs = np.round(outputs)

                outputs[outputs>THRESHOLD] = 1 
                outputs[outputs<=THRESHOLD] = 0
                #outputs = torch.round(outputs)

                #Aggregated per-pixel loss
                #loss = criterion(outputs, mask)
                
                loss = criterion(outputs.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0).unsqueeze(0))
                loss_per_batch.append(loss.item())

                # Convert to numpy
                outputs = outputs.detach().cpu().numpy()
                outputs = outputs.astype("int64")
                #predictions = np.round(predictions)

                mask = mask.data.cpu().numpy()
                mask = mask.astype("int64")

                '''
                import matplotlib.pyplot as plt
                plt.imshow(outputs)
                np.unique(outputs)

                import seaborn as sns
                sns.histplot(data = outputs)

                preds = outputs.data.max(1)[1].squeeze(1).squeeze(0).cpu().numpy()

                mask = mask.data.cpu().numpy()
                mask = mask.astype("int64")
                plt.imshow(mask)

                assert np.unique(mask) == np.unique(outputs*255)
                '''
                gts_all.append(mask)
                predictions_all.append(255*outputs)

                if SAVE:

                    images = "outputs"
                    labels = images+"/predictions"
                    true = images+"/labels"

                    if not os.path.exists(images):
                        os.makedirs(images)
                    if not os.path.exists(labels):
                            os.makedirs(labels)
                    if not os.path.exists(true):
                            os.makedirs(true)

                    matplotlib.image.imsave(f"{labels}/Pred_{batch_idx}.png", outputs , cmap='gray')

                    matplotlib.image.imsave(f"{true}/True_{batch_idx}.png", mask, cmap='gray')
            else:
                break


    #test_loss /= len(test_loader.dataset)
    loss_per_epoch = [np.average(loss_per_batch)]
    

    ##Compute Mean Intersection over Union (mIoU)
    ##mIoU: Mean (of all classes) of intersection over union between prediction
    ##and ground-truth

    # Improved: https://towardsdatascience.com/intersection-over-union-iou-calculation-for-evaluating-an-image-segmentation-model-8b22e2e84686 
    iou_scores = []
    for lp, lt in zip(predictions_all, gts_all):
        # Convert lp to cpu
        #lp = lp.detach().cpu().numpy()
        intersection = np.logical_and(lp.flatten(), lt.flatten())  
        union = np.logical_or(lp.flatten(), lt.flatten())  
        iou_score = np.sum(intersection) / np.sum(union)
        iou_scores.append(iou_score)
    
    mean_iu = np.average(iou_scores)
    
    print('\nTest set ({:.0f}): Average loss: {:.4f}, mIoU: {:.4f}\n'.format(
        len(test_loader.dataset), loss_per_epoch[-1], mean_iu)) 
    
    ###########################################################################
    # Store predictions

    return loss_per_epoch, mean_iu, predictions_all, gts_all
    

model = unet_bas().to(device)

criterion = FocalDiceLoss()# nn.BCEWithLogitsLoss()
print('Total params: %2.fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))
    
milestones = args.M
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-4)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

import os
import time
loss_train_epoch = []
loss_test_epoch = []
acc_train_per_epoch = []
acc_test_per_epoch = []
new_labels = []

cont = 0

res_path = "./metrics_" + args.experiment_name


model_path = "./models" # + filename

if not os.path.exists(model_path):
    os.makedirs(model_path)
    print("models folder created")

if LOAD_MODEL:
        load_checkpoint_pspnetlite(torch.load("models/" + filename), model)
    

if not os.path.isdir(res_path):
    os.makedirs(res_path)


for epoch in range(1, args.epoch + 1):
        st = time.time()
        scheduler.step()    
        # train for one epoch
        if TRAIN:

            print("Unet improved --> training, epoch " + str(epoch))
    
            loss_per_epoch = train_SemanticSeg(args, model, device, train_loader, optimizer, epoch, criterion)
    
            loss_train_epoch += [loss_per_epoch]
            
            if epoch == args.epoch:
                torch.save(model.state_dict(), "models/" + filename)
    
            np.save(res_path + '/' + 'LOSS_epoch_train.npy', np.asarray(loss_train_epoch))
        
        # TESTING
        if TEST:
            print("Unet improved ==> testing, epoch " + str(epoch))
            # test
            loss_per_epoch_test, acc_val_per_epoch_i, a,b = testing(args, model, device, test_loader, criterion)
    
            loss_test_epoch += loss_per_epoch_test
            acc_test_per_epoch += [acc_val_per_epoch_i]
    
    
            if epoch == 1:
                best_acc_val = acc_val_per_epoch_i
    
            else:
                if acc_val_per_epoch_i > best_acc_val:
                    best_acc_val = acc_val_per_epoch_i
    
    
            np.save(res_path + '/' + 'LOSS_epoch_val.npy', np.asarray(loss_test_epoch))
    
            # save accuracies:
            np.save(res_path + '/' + 'accuracy_per_epoch_val.npy', np.asarray(acc_test_per_epoch))
            
            cont += 1

    
# PREDICT

if PREDICT: 
    
    y_pred, true_pred = [], []
    
    loss_per_epoch_test, acc_val_per_epoch_i, y_pred, true_pred = testing(args, model, device, test_loader, criterion)
    
    y_pred = np.array(y_pred)
    true_pred = np.array(true_pred)
    
    print(f"y_pred sizes: {y_pred.shape} - true_pred sizes: {true_pred.shape}")
    print(f"y_pred pixel values: {np.unique(y_pred)}")

    store_predictions_with_patching(y_pred, true_pred, 16)


##Accuracy
acc_test = np.load(res_path + '/' + 'accuracy_per_epoch_val.npy')

#Loss per epoch
loss_train = np.load(res_path + '/' + 'LOSS_epoch_train.npy')
loss_test = np.load(res_path + '/' + 'LOSS_epoch_val.npy')

numEpochs = len(acc_test)
epochs = range(numEpochs)

plt.figure(1)
plt.plot(epochs, acc_test, label='Test, max acc: ' + str(np.max(acc_test)))
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

plt.figure(2)
plt.plot(epochs, loss_test, label='Test, min loss: ' + str(np.min(loss_test)))
plt.plot(epochs, loss_train, label='Train, min loss: ' + str(np.min(loss_train)))
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.show()


PyTorch version: 1.10.0
Total params: 31M
Unet improved --> training, epoch 1


In [ ]:
!ls